In [1]:
# persistent execution state
# dynamic interrupts
# static interrupts
# flexible integration points

# pattern 
# approve or reject
# edit graph state
# review tool call
# validate human input 

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

### pause using intrrupt and Command

In [ ]:
from typing import TypedDict
import uuid
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph

from langgraph.types import interrupt, Command

class State(TypedDict):
    some_text: str


def human_node(state: State):
    value = interrupt(  
        {
            "text_to_revise": state["some_text"]  
        }
    )
    return {
        "some_text": value  
    }


# Build the graph
graph_builder = StateGraph(State)
graph_builder.add_node("human_node", human_node)
graph_builder.add_edge(START, "human_node")
checkpointer = InMemorySaver()  
graph = graph_builder.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": uuid.uuid4()}}
result = graph.invoke({"some_text": "original text"}, config=config)

In [8]:
print(graph.invoke(Command(resume="Edited text asd"), config=config)) 

{'some_text': 'Edited text asd'}


### Resume multiple interrupts with one invocation

In [21]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

class CustomState(TypedDict):
    text_1: str
    text_2: str

def node_1(state: CustomState):
    value = interrupt({"text_to_revise": state["text_1"]})
    return {"text_1": value}

def node_2(state: CustomState):
    value = interrupt({"text_to_revise": state["text_2"]})
    return {"text_2": value}

workflow = (
    StateGraph(CustomState)
    .add_node("node_1", node_1)
    .add_node("node_2", node_2)
    .add_edge(START, "node_1")
    .add_edge(START, "node_2")
    .compile(checkpointer=checkpointer)
)

import uuid

config = {
    "configurable":{
        "thread_id": str(uuid.uuid4())
    }
}

In [22]:
result = workflow.invoke({"text_1":"sample 1", "text_2":"sample 2"}, config=config)

In [23]:
workflow.get_state(config).interrupts

(Interrupt(value={'text_to_revise': 'sample 1'}, resumable=True, ns=['node_1:02cc6dd3-0ff9-4323-a2c0-cd5fcc489234']),
 Interrupt(value={'text_to_revise': 'sample 2'}, resumable=True, ns=['node_2:4016d254-115b-d0b7-3849-f0e7a0a231ef']))

In [18]:
resume_map = {
    i.interrupt_id: f"human input for prompt {i.value["text_to_revise"]}"
    for i in workflow.get_state(config).interrupts
}

resume_map

{'bce26acd4a2e361e4a32b1997bec4e99': 'human input for prompt sample 1',
 '170065d56cfa3c5ce7d67c5baf60fdd1': 'human input for prompt sample 2'}

In [20]:
print(workflow.invoke(Command(resume=resume_map), config=config))

{'text_1': 'human input for prompt sample 1', 'text_2': 'human input for prompt sample 2'}


### Comman Pattern 

#### Approve and reject

In [ ]:
from langgraph.types import interrupt, Command
import uuid
from langgraph.constants import START, END
from langgraph.graph import StateGraph, MessagesState
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict, Literal

checkpoint = InMemorySaver()

class CustomState(TypedDict):
    llm_output: str
    decision: str

def generate_llm_output(state: CustomState) -> CustomState:
    return {"llm_output": state["llm_output"]}

def human_approval(state: CustomState) -> Command[Literal["approved_path", "rejected_path"]]:
    decision = interrupt({
        "question": "Do you approve the following output?",
        "llm_output": state["llm_output"]
    })

    if decision == "approve":
        return Command(goto="approved_path", update={"decision": "approve"})
    else:
        return Command(goto="rejected_path", update={"decision": "reject"})
    
def approved_path(state: CustomState) -> CustomState:
    print(f"Approve node : {state["decision"]}")
    return state
        
def rejected_path(state: CustomState) -> CustomState:
    print(f"Reject node : {state["decision"]}")
    return state

workflow = (
    StateGraph(CustomState)
    .add_node("generate_llm_output", generate_llm_output)
    .add_node("human_approval", human_approval)
    .add_node("approved_path", approved_path)
    .add_node("rejected_path", rejected_path)
    .add_edge(START, "generate_llm_output")
    .add_edge("generate_llm_output", "human_approval")
    .add_edge("approved_path", END)
    .add_edge("rejected_path", END)
    .compile(checkpointer=checkpoint)
)

import uuid

config = {
    "configurable":{
        "thread_id": str(uuid.uuid4())
    }
}

config

{'configurable': {'thread_id': '61887ee0-f0b8-42c0-b580-aec3912718f3'}}

In [7]:
response = workflow.invoke({"llm_output":"hello"}, config=config)
response

{'llm_output': 'hello',
 '__interrupt__': [Interrupt(value={'question': 'Do you approve the following output?', 'llm_output': 'hello'}, resumable=True, ns=['human_approval:df80d458-a392-0b04-a0fa-7bc311a15b1c'])]}

In [10]:
final_result = workflow.invoke(Command(resume="approve"), config=config)
final_result

Approve node : approve


{'llm_output': 'hello', 'decision': 'approve'}

#### Review and edit state

In [12]:
from langgraph.graph import StateGraph
from langgraph.constants import START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import Literal, TypedDict

class CustomState(TypedDict):
    llm_output: str

checkpoint = InMemorySaver()

def generate_llm_output(state: CustomState) -> CustomState:
    return {"llm_output": state["llm_output"]}

def human_edit_node(state: CustomState) -> CustomState:
    response = interrupt({
        "request": "make simple edit",
        "llm_generate_output": state["llm_output"]
    })

    return {
        "llm_output": response["change_output"]
    }

def checking_node(state: CustomState) -> CustomState:
    print(f"Check Change Output: {state['llm_output']}")
    return state

workflow = (
    StateGraph(CustomState)
    .add_node("generate_llm_output", generate_llm_output)
    .add_node("human_edit_node", human_edit_node)
    .add_node("checking_node", checking_node)
    .set_entry_point("generate_llm_output")
    .add_edge("generate_llm_output", "human_edit_node")
    .add_edge("human_edit_node", "checking_node")
    .compile(checkpointer=checkpoint)
)

import uuid

config = {
    "configurable":{
        "thread_id": str(uuid.uuid4())
    }
}

config

{'configurable': {'thread_id': '6f331fb0-20cd-400b-ba24-4b250e67e588'}}

In [13]:
respose = workflow.invoke({"llm_output": "demo llm output"}, config=config)
respose

{'llm_output': 'demo llm output',
 '__interrupt__': [Interrupt(value={'request': 'make simple edit', 'llm_generate_output': 'demo llm output'}, resumable=True, ns=['human_edit_node:d49cdce0-36c7-33d8-0f55-0448e83c689e'])]}

In [14]:
final_edit = workflow.invoke(Command(resume={"change_output": "llm output is change"}), config=config)
final_edit

Check Change Output: llm output is change


{'llm_output': 'llm output is change'}

#### Review Tool calls

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.types import interrupt
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command, interrupt

checkpoint = InMemorySaver()

model = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", api_key=GEMINI_API_KEY)

def check_hotel_name(hotel_name: str):
    """check hotel name

    Args:
        hotel_name: name of hotel
    """
    response = interrupt({
        f"Call hotel_name function with {hotel_name}, edit or approve"
    })

    if response["type"] == "edit":
        hotel_name = response["args"]["hotel_name"]
    elif response["type"] == "accept":
        pass
    else:
        raise ValueError(f"Unknown response type: {response['type']}")
    
    return f"Successfully booked a stay at {hotel_name}."

agent = create_react_agent(
    model=model,
    tools=[check_hotel_name],
    checkpointer=checkpoint
)

import uuid

config = {
    "configurable":{
        "thread_id": str(uuid.uuid4())
    }
}

config


{'configurable': {'thread_id': 'fa943d44-ec2a-4305-8eaf-a07337f6242e'}}

In [26]:
response = agent.invoke({"messages": [{"role": "user", "content": "book a stay at McKittrick hotel"}]},config=config)
response

{'messages': [HumanMessage(content='book a stay at McKittrick hotel', additional_kwargs={}, response_metadata={}, id='172c5200-b00f-4e2f-94ce-54e561674403'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'check_hotel_name', 'arguments': '{"hotel_name": "McKittrick hotel"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4f8eb1f6-20b5-4caa-a92c-6870e29f12d1-0', tool_calls=[{'name': 'check_hotel_name', 'args': {'hotel_name': 'McKittrick hotel'}, 'id': '8268c3dd-cd66-4d8a-888e-5844ff603b20', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 88, 'total_tokens': 151, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 66}})],
 '__interrupt__': [Interrupt(value={'Call hotel_name function with McKittrick hotel, edit or approve'}, resumable=True, ns=['tools:4dd86363-32f5-9d24-d93b-b58eb8e686c3

In [27]:
# response = agent.invoke(Command(resume={"type":"accept"}),config=config)
response = agent.invoke(Command(resume={"type":"edit", "args":{"hotel_name":"creditt"}}),config=config)
response

{'messages': [HumanMessage(content='book a stay at McKittrick hotel', additional_kwargs={}, response_metadata={}, id='172c5200-b00f-4e2f-94ce-54e561674403'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'check_hotel_name', 'arguments': '{"hotel_name": "McKittrick hotel"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4f8eb1f6-20b5-4caa-a92c-6870e29f12d1-0', tool_calls=[{'name': 'check_hotel_name', 'args': {'hotel_name': 'McKittrick hotel'}, 'id': '8268c3dd-cd66-4d8a-888e-5844ff603b20', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 88, 'total_tokens': 151, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 66}}),
  ToolMessage(content='Successfully booked a stay at creditt.', name='check_hotel_name', id='56d72cbe-6df2-48e5-b2bf-b206493333b0', tool_call_id='8268c3dd-cd66-4d8a-888

#### Add interrupt to any tool

In [45]:
# add interrupt to any tool
from typing import Callable
from langchain_core.tools import BaseTool, tool as create_tool
from langchain_core.runnables import RunnableConfig
from langgraph.types import interrupt
from langgraph.prebuilt.interrupt import HumanInterruptConfig, HumanInterrupt

def add_human_in_loop(tool: Callable | BaseTool, *, interrupt_config: HumanInterruptConfig = None) -> BaseTool:
    """Wrap a tool to support human-in-the-loop review."""
    if not isinstance(tool, BaseTool):
        tool = create_tool(tool)

    if interrupt_config == None:
        interrupt_config ={
            "allow_accept": True,
            "allow_edit": True,
            "allow_respond": True
        }

    @create_tool(tool.name, description=tool.description, args_schema=tool.args_schema)
    def call_tool_with_interrupt(config: RunnableConfig, **tool_input):
        request: HumanInterrupt = {
            "action_request" :{
                "action": tool.name,
                "args": tool_input
            },
            "config": interrupt_config,
            "description": "please review tool call"
        }

        response = interrupt([request])[0]

        if response["type"] == "edit":
            print(response)
            tool_input = response["args"]["args"]
            tool_response = tool.invoke(tool_input, config=config)
        elif response["type"] == "accept":
            tool_response = tool.invoke(tool_input, config=config)
        elif response["type"] == "response":
            user_feedback  = response["args"]
            tool_response = user_feedback


        return tool_response
    
    return call_tool_with_interrupt
    

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI

checkpoint = InMemorySaver()

def book_hotel(hotel_name: str):
   """Book a hotel

    Args:
        hotel_name: name of hotel
   """
   return f"Successfully booked a stay at {hotel_name}."

model = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", api_key=GEMINI_API_KEY)

agent = create_react_agent(
    model=model,
    tools=[add_human_in_loop(book_hotel)],
    checkpointer=checkpoint
)

config = {"configurable": {"thread_id": "1"}}

In [52]:
response = agent.invoke({"messages": [{"role": "user", "content": "book a stay at McKittrick hotel"}]},config=config)
response

{'messages': [HumanMessage(content='book a stay at McKittrick hotel', additional_kwargs={}, response_metadata={}, id='4f9fd17c-7845-424c-bbed-73234e84ef4f'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'book_hotel', 'arguments': '{"hotel_name": "McKittrick hotel"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4021c750-b090-4da3-8a08-10e87dfe2eab-0', tool_calls=[{'name': 'book_hotel', 'args': {'hotel_name': 'McKittrick hotel'}, 'id': 'fa2fa4cc-02cc-4bf0-ba29-e3fd903295ab', 'type': 'tool_call'}], usage_metadata={'input_tokens': 61, 'output_tokens': 95, 'total_tokens': 156, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 75}})],
 '__interrupt__': [Interrupt(value=[{'action_request': {'action': 'book_hotel', 'args': {'hotel_name': 'McKittrick hotel'}}, 'config': {'allow_accept': True, 'allow_edit': True, '

In [53]:
response = agent.invoke(Command(resume={"type":"accept"}),config=config)
# response = agent.invoke(Command(resume={"type":"edit", "args":{"hotel_name":"creditt++++"}}),config=config)
response

{'messages': [HumanMessage(content='book a stay at McKittrick hotel', additional_kwargs={}, response_metadata={}, id='4f9fd17c-7845-424c-bbed-73234e84ef4f'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'book_hotel', 'arguments': '{"hotel_name": "McKittrick hotel"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4021c750-b090-4da3-8a08-10e87dfe2eab-0', tool_calls=[{'name': 'book_hotel', 'args': {'hotel_name': 'McKittrick hotel'}, 'id': 'fa2fa4cc-02cc-4bf0-ba29-e3fd903295ab', 'type': 'tool_call'}], usage_metadata={'input_tokens': 61, 'output_tokens': 95, 'total_tokens': 156, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 75}}),
  ToolMessage(content='Error: KeyError(0)\n Please fix your mistakes.', name='book_hotel', id='c682778a-9328-45a3-be9e-91b7e41942c8', tool_call_id='fa2fa4cc-02cc-4bf0-ba29-e3fd9032

#### Validate human input

In [56]:
from langgraph.graph import StateGraph
from langgraph.constants import START, END
from langgraph.types import interrupt, Command
import uuid
from langgraph.checkpoint.memory import InMemorySaver
from typing import Literal, TypedDict

checkpoint = InMemorySaver()

class CustomState(TypedDict):
    age: int

def get_validate_age(state: CustomState) -> CustomState:
    prompt = "please enter youar age(must be integer)"

    while True:
        user_input = interrupt(prompt)

        try:
            age = int(user_input)
            if age < 0:
                raise ValueError("age is negative, enter positive integer")
            break
        except (ValueError, TypeError):
            prompt = f"'{user_input}' is not valid. Please enter a non-negative integer for age."

    return {'age': age}

def report_age(state: CustomState) -> CustomState:
    print(f"✅ Human is {state['age']} years old.")
    return state


workflow = (
    StateGraph(CustomState)
    .add_node("get_validate_age", get_validate_age)
    .add_node("report_age", report_age)
    .set_entry_point("get_validate_age")
    .add_edge("get_validate_age", "report_age")
    .add_edge("report_age", END)
    .compile(checkpointer=checkpoint)
)

import uuid

config = {
    "configurable":{
        "thread_id": str(uuid.uuid4())
    }
}
config

{'configurable': {'thread_id': '2cf8c7d5-6015-4dc0-b0d2-335d9357274d'}}

In [57]:
result = workflow.invoke({}, config=config)
result

{'__interrupt__': [Interrupt(value='please enter youar age(must be integer)', resumable=True, ns=['get_validate_age:7fdb63b4-a400-e6f8-f590-afdbd600fb01'])]}

In [58]:
result = workflow.invoke(Command(resume="not a number"), config=config)
result

{'__interrupt__': [Interrupt(value="'not a number' is not valid. Please enter a non-negative integer for age.", resumable=True, ns=['get_validate_age:7fdb63b4-a400-e6f8-f590-afdbd600fb01'])]}

In [59]:
result = workflow.invoke(Command(resume="-1"), config=config)
result

{'__interrupt__': [Interrupt(value="'-1' is not valid. Please enter a non-negative integer for age.", resumable=True, ns=['get_validate_age:7fdb63b4-a400-e6f8-f590-afdbd600fb01'])]}

In [60]:
result = workflow.invoke(Command(resume="5"), config=config)
result

✅ Human is 5 years old.


{'age': 5}

#### Debug with interrupts

In [64]:
from langgraph.graph import StateGraph, MessagesState
from langgraph.types import interrupt, Command
from langgraph.constants import START, END
from langgraph.checkpoint.memory import InMemorySaver

checkpoint = InMemorySaver()

def step_1(state: MessagesState) -> MessagesState:
    return {"messages":f"hello world, {state["messages"]}"}

def step_2(state: MessagesState) -> MessagesState:
    print("Node 2")
    return {"messages": f"node 2 is execute"}

def step_3(state: MessagesState) -> MessagesState:
    print("Node 3")
    return {"messages": f"node 3 is execute"}

workflow = (
    StateGraph(MessagesState)
    .add_node("step_1", step_1)
    .add_node("step_2", step_2)
    .add_node("step_3", step_3)
    .set_entry_point("step_1")
    .add_edge("step_1", "step_2")
    .add_edge("step_2", "step_3")
    .add_edge("step_3", END)
    .compile(
        checkpointer=checkpoint,
        interrupt_after=["step_1"],
        interrupt_before=["step_2", "step_3"]
    )
)

import uuid

config = {
    "configurable":{
        "thread_id": str(uuid.uuid4())
    }
}
config

{'configurable': {'thread_id': '872b255a-469d-436e-b2e8-f93f21fb2519'}}

In [65]:
response = workflow.invoke({"messages": "hello"}, config=config)
response

{'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='b7257382-acc5-4c1b-9d8a-d5cf4c10c757'),
  HumanMessage(content="hello world, [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='b7257382-acc5-4c1b-9d8a-d5cf4c10c757')]", additional_kwargs={}, response_metadata={}, id='9ec67c74-5c19-4bd4-a350-c0ed845802a1')]}

In [66]:
response = workflow.invoke(None, config=config)
response

Node 2


{'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='b7257382-acc5-4c1b-9d8a-d5cf4c10c757'),
  HumanMessage(content="hello world, [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='b7257382-acc5-4c1b-9d8a-d5cf4c10c757')]", additional_kwargs={}, response_metadata={}, id='9ec67c74-5c19-4bd4-a350-c0ed845802a1'),
  HumanMessage(content='node 2 is execute', additional_kwargs={}, response_metadata={}, id='1f7d01c5-72fa-483c-99e9-3f2f19724121')]}

In [67]:
response = workflow.invoke(None, config=config)
response

Node 3


{'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='b7257382-acc5-4c1b-9d8a-d5cf4c10c757'),
  HumanMessage(content="hello world, [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='b7257382-acc5-4c1b-9d8a-d5cf4c10c757')]", additional_kwargs={}, response_metadata={}, id='9ec67c74-5c19-4bd4-a350-c0ed845802a1'),
  HumanMessage(content='node 2 is execute', additional_kwargs={}, response_metadata={}, id='1f7d01c5-72fa-483c-99e9-3f2f19724121'),
  HumanMessage(content='node 3 is execute', additional_kwargs={}, response_metadata={}, id='3423d24a-dd0b-495d-babd-840ec5f36e43')]}

In [ ]:
# considerations 
# 1. side effect, call api or execute function after inttrupt or make new node 
# 2. when subgraph have intrrupts then parent graph execute from interrupt or subgraph called node
# 3. mulitple inttrupts in single node